In [ ]:
### código para descargar los pdfs de los enlaces de cada contrato en la página de contratación de México. Descarga todos los archivos en pdf y crea uan carpeta con el nombre del expediente apra guardarlos################

In [ ]:
{"_id":"anexos","startUrl":["https://compranet.hacienda.gob.mx/esop/toolkit/opportunity/ns/1945767/detail.si?isOnModification=false&_ncp=1684331590515.969733-2#fh"],"selectors":[{"id":"anexo","multiple":true,"parentSelectors":["_root"],"selector":"td > a","type":"SelectorLink"},{"id":"titulo","multiple":false,"parentSelectors":["_root"],"regex":"","selector":"div.subtitle_01","type":"SelectorText"}]}


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from highlight_sel_element import highlight
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import re
import time
#import PyPDF2
import collections
import requests
import threading
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException


In [9]:
############ código para descargar anexos ################

df = pd.read_excel('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2021.xlsx')
carpeta_descargas = os.path.expanduser('/Users/datos-lc/Downloads/')

no_descargados = []  # Lista para almacenar los posibles errores
descargados = []  # Lista para almacenar los contratos descargados

### for que usa dos parámetros del dataframe. Para cada una de esas columnas hace las búsquedas y descargas.
for columna1, columna2 in zip(df['Código del expediente'], df['Dirección del anuncio']):
   
    # Número del expediente que aparece en la columna del excel, para comprobar después que sí sea ese el contrato descargado
    expediente_excel = str(columna1)

    # Navegar en este enlace
    driver = webdriver.Firefox()
    driver.get(columna2)

    # Obtener el titulo con información del expediente y nombre del contrato que aparece en la página, sirve para nombrar la carpeta con los PDFs
    try:
        titulo = driver.find_element(By.XPATH, "//div[@class='subtitle_01 maintitle']")
        nombre_carpeta_contrato = titulo.text.strip()
        # Ruta completa de la carpeta para los archivos del contrato
        ruta_contrato = os.path.join(f'/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2021/', nombre_carpeta_contrato)

        # Crear la carpeta con el nombre del contrato para meter los archivos individuales
        if not os.path.exists(ruta_contrato):
            os.makedirs(ruta_contrato)
            
        ruta_anexo = os.path.join(ruta_contrato, "- Anexo")

        #Crear la lista con los elementos descargables
        pdf = driver.find_elements(By.XPATH, "//table[2]/tbody/tr[2]/td[2]/a")
        cantidad_pdf = len(pdf)
        print(cantidad_pdf)
        
        for archivo in pdf:   
            archivo.click()
            time.sleep(10)
        
    except StaleElementReferenceException:
            print("Elemento obsoleto. Se omitirá y continuará con el siguiente.")
            print(nombre_carpeta_contrato)
            print(columna2)
            no_descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
            driver.quit()
            continue
              
              
    files = os.listdir(carpeta_descargas)
    
    if len(files)> 0:
    
        for descargado in files:
            ruta_vieja = os.path.join(carpeta_descargas, descargado)
            ruta_nueva = os.path.join(ruta_contrato, descargado) 
            os.rename(ruta_vieja, ruta_nueva)
            descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, descargado])

    else:
        print("No se encontraron archivos en la carpeta de descargas.")
       

    driver.quit()


df_no_descargados = pd.DataFrame(no_descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina"])
df_descargados = pd.DataFrame(descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina", "Nombre_archivo_descargado"])


16


In [ ]:
############ codigo para descargar elementos faltantes en anterior iteración ################

df = pd.read_excel('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/Excel/2021.xlsx')
carpeta_descargas = os.path.expanduser('/Users/datos-lc/Downloads/')

no_descargados = []  # Lista para almacenar los posibles errores
descargados = []  # Lista para almacenar los contratos descargados

### for que usa dos parámetros del dataframe. Para cada una de esas columnas hace las búsquedas y descargas.
for columna1, columna2 in zip(df['Código del expediente'], df['Dirección del anuncio']):
   
    # Número del expediente que aparece en la columna del excel, para comprobar después que sí sea ese el contrato descargado
    expediente_excel = str(columna1)

    # Navegar en este enlace
    driver = webdriver.Firefox()
    driver.get(columna2)

    # Obtener el titulo con información del expediente y nombre del contrato que aparece en la página, sirve para nombrar la carpeta con los PDFs
    titulo = driver.find_element(By.XPATH, "//div[@class='subtitle_01 maintitle']")
    nombre_carpeta_contrato = titulo.text.strip()

    # Ruta completa de la carpeta para los archivos del contrato
    ruta_contrato = os.path.join(f'/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2021/', nombre_carpeta_contrato)
    


    if os.path.exists(ruta_contrato):
        
        print(f"la carpeta{nombre_carpeta_contrato} ya tiene contenido")
        print(" ------------------------------------------------------")
        driver.quit()
        
    else:
        os.makedirs(ruta_contrato)
        print(f"la carpeta{nombre_carpeta_contrato} NO EXISTEEEEEEEEEEE") 
        print(" ------------------------------------------------------")
       # Leer el número del expediente que aparece en la página web que estamos navegando
        aux = driver.find_element(By.XPATH, "//div[@class='form_container'][1]/ul/li/div[@class='form_answer']")
        expediente_pagina = aux.text.strip()

        #Crear la lista con los elementos descargables
        pdf = driver.find_elements(By.XPATH, "//a[@class='attachTaglink']")
        cantidad_pdf = len(pdf)
        print(cantidad_pdf)

        for archivo in pdf:
            try:
                archivo.click()
                time.sleep(10)
            except StaleElementReferenceException:
                print("Elemento obsoleto. Se omitirá y continuará con el siguiente.")
                print(nombre_carpeta_contrato)
                print(columna2)
                no_descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
                continue

        files = os.listdir(carpeta_descargas)

        if len(files)> 0:

            for descargado in files:
                ruta_vieja = os.path.join(carpeta_descargas, descargado)
                ruta_nueva = os.path.join(ruta_contrato, descargado) 
                os.rename(ruta_vieja, ruta_nueva)
                descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, descargado])

        else:
            print("No se encontraron archivos en la carpeta de descargas.")
            no_descargados.append([nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])

        driver.quit()


df_no_descargados = pd.DataFrame(no_descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina"])
df_descargados = pd.DataFrame(descargados, columns=["Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina", "Nombre_archivo_descargado"])


In [67]:
def sanitize_filename(filename):
    """
    Reemplaza los caracteres no válidos en nombres de archivos en macOS.
    """
    invalid_chars = r'[/\\?*:"><|]'
    sanitized_filename = re.sub(invalid_chars, '_', filename)
    return sanitized_filename


In [84]:
######################## Leer cada pestaña del archivo Excel ##################################
#En este caso, pd.read_excel() se utiliza con el argumento sheet_name=None, 
#lo que significa que todas las pestañas del archivo se leerán y se almacenarán en un diccionario de DataFrames, 
#donde las claves son los nombres de las pestañas y los valores son los DataFrames correspondientes. 

"""
En el código for nombre_pestaña, df in dataframes.items(), se utilizan dos variables, nombre_pestaña y df, 
para desempaquetar los elementos del diccionario dataframes.items(). 
La variable nombre_pestaña contendrá la clave (nombre de la pestaña) y la variable df contendrá el valor (DataFrame) 
correspondiente a esa clave.

"""

dataframes = pd.read_excel('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/Contratos Guardia Nacional.xlsx', sheet_name=None)

no_descargados = []  # Lista para almacenar los posibles errores
descargados = []  # Lista para almacenar los contratos descargados

# Leer cada pestaña del archivo Excel
for nombre_pestaña, df in dataframes.items():
    # Obtener el nombre de la carpeta (nombre de la pestaña)
    nombre_carpeta_year = nombre_pestaña.strip()

    # Ruta completa donde deseas crear la carpeta de los years
    ruta_carpeta_year = os.path.join('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/', nombre_carpeta_year)

    # Crear la carpeta si no existe
    if not os.path.exists(ruta_carpeta_year):
        os.makedirs(ruta_carpeta_year)
        print("Carpeta creada:", ruta_carpeta_year)
    else:
        print("La carpeta ya existe:", ruta_carpeta_year)

    time.sleep(2)

    ### for que usa dos parámetros del dataframe. Para cada una de esas columnas hace las búsquedas y descargas.
    for columna1, columna2 in zip(df['Código del expediente'], df['Dirección del anuncio']):
        # Número del expediente que aparece en la columna del excel, para comprobar después que sí sea ese el contrato descargado
        expediente_excel = str(columna1)
        
        # Navegar en este enlace
        driver = webdriver.Chrome()
        driver.get(columna2)

        # Obtener el titulo con información del expediente y nombre del contrato que aparece en la página, sirve para nombrar la carpeta con los PDFs
        titulo = driver.find_element(By.XPATH, "//div[@class='subtitle_01 maintitle']")
        nombre_carpeta_contrato = titulo.text.strip()

        # Ruta completa de la carpeta para los archivos del contrato
        ruta_completa_contrato = os.path.join(f'/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/{nombre_carpeta_year}/', nombre_carpeta_contrato)

        # Crear la carpeta con el nombre del contrato para meter los archivos individuales
        if not os.path.exists(ruta_completa_contrato):
            os.makedirs(ruta_completa_contrato)

        # Leer el número del expediente que aparece en la página web que estamos navegando
        aux = driver.find_element(By.XPATH, "//div[@class='form_container'][1]/ul/li/div[@class='form_answer']")
        expediente_pagina = aux.text.strip()
        #Crear la lista con los elementos descargables
        pdf = driver.find_elements(By.XPATH, "//a[@class='attachTaglink']")
        cantidad_pdf = len(pdf)


        for elemento in pdf:
            nombre_archivo_pdf = elemento.text.strip()
            elemento.click()
            time.sleep(7)

            carpeta_descargas = os.path.expanduser('/Users/datos-lc/Downloads/')
            pdf_files = [f for f in os.listdir(carpeta_descargas) if f.endswith(".pdf")]

            if len(pdf_files) == 0:
                print("No se encontraron archivos en la carpeta de descargas.")
                no_descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
            else:

                name_descarga = files[0]
                os.rename(os.path.join(carpeta_descargas, name_descarga), os.path.join(ruta_completa_contrato, nombre_archivo_pdf))
                descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, name_descarga, nombre_archivo_pdf])
        driver.quit()

    
    """ except Exception as e:
            

            print("Error en el try mayor", e)
            print(nombre_archivo_pdf)
            no_descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
            #driver.quit()
            time.sleep(2)
            continue"""

        
df_no_descargados = pd.DataFrame(no_descargados, columns=["Year","Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina"])
df_descargados = pd.DataFrame(descargados, columns=["Year","Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina", "Nombre_archivo_descargod_original", "Nombre_nuevo_archivo_en_carpeta"])
                      





Carpeta creada: /Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2019


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.92)
Stacktrace:
0   chromedriver                        0x000000010e285598 chromedriver + 4973976
1   chromedriver                        0x000000010e27c913 chromedriver + 4938003
2   chromedriver                        0x000000010de39cd6 chromedriver + 470230
3   chromedriver                        0x000000010de0f58c chromedriver + 296332
4   chromedriver                        0x000000010dea6f0f chromedriver + 917263
5   chromedriver                        0x000000010debbef6 chromedriver + 1003254
6   chromedriver                        0x000000010dea1793 chromedriver + 894867
7   chromedriver                        0x000000010de6f21f chromedriver + 688671
8   chromedriver                        0x000000010de7056e chromedriver + 693614
9   chromedriver                        0x000000010e23fbb7 chromedriver + 4688823
10  chromedriver   

In [71]:
######################## Leer cada pestaña del archivo Excel ##################################
#En este caso, pd.read_excel() se utiliza con el argumento sheet_name=None, 
#lo que significa que todas las pestañas del archivo se leerán y se almacenarán en un diccionario de DataFrames, 
#donde las claves son los nombres de las pestañas y los valores son los DataFrames correspondientes. 

"""
En el código for nombre_pestaña, df in dataframes.items(), se utilizan dos variables, nombre_pestaña y df, 
para desempaquetar los elementos del diccionario dataframes.items(). 
La variable nombre_pestaña contendrá la clave (nombre de la pestaña) y la variable df contendrá el valor (DataFrame) 
correspondiente a esa clave.

"""

dataframes = pd.read_excel('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/Contratos Guardia Nacional.xlsx', sheet_name=None)

no_descargados = []  # Lista para almacenar los posibles errores
descargados = []  # Lista para almacenar los contratos descargados

# Leer cada pestaña del archivo Excel
for nombre_pestaña, df in dataframes.items():
    # Obtener el nombre de la carpeta (nombre de la pestaña)
    nombre_carpeta_year = nombre_pestaña.strip()

    # Ruta completa donde deseas crear la carpeta de los years
    ruta_carpeta_year = os.path.join('/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/', nombre_carpeta_year)

    # Crear la carpeta si no existe
    if not os.path.exists(ruta_carpeta_year):
        os.makedirs(ruta_carpeta_year)
        print("Carpeta creada:", ruta_carpeta_year)
    else:
        print("La carpeta ya existe:", ruta_carpeta_year)

    time.sleep(2)

    ### for que usa dos parámetros del dataframe. Para cada una de esas columnas hace las búsquedas y descargas.
    for columna1, columna2 in zip(df['Código del expediente'], df['Dirección del anuncio']):
        # Número del expediente que aparece en la columna del excel, para comprobar después que sí sea ese el contrato descargado
        expediente_excel = str(columna1)


        # Navegar en este enlace
        driver = webdriver.Chrome()
        driver.get(columna2)

        # Obtener el titulo con información del expediente y nombre del contrato que aparece en la página, sirve para nombrar la carpeta con los PDFs
        titulo = driver.find_element(By.XPATH, "//div[@class='subtitle_01 maintitle']")
        nombre_carpeta_contrato = titulo.text.strip()

        # Ruta completa de la carpeta para los archivos del contrato
        ruta_completa_contrato = os.path.join(f'/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/{nombre_carpeta_year}/', nombre_carpeta_contrato)

        # Crear la carpeta con el nombre del contrato para meter los archivos individuales
        if not os.path.exists(ruta_completa_contrato):
            os.makedirs(ruta_completa_contrato)

        # Leer el número del expediente que aparece en la página web que estamos navegando
        aux = driver.find_element(By.XPATH, "//div[@class='form_container'][1]/ul/li/div[@class='form_answer']")
        expediente_pagina = aux.text.strip()
        #Crear la lista con los elementos descargables
        pdf = driver.find_elements(By.XPATH, "//a[@class='attachTaglink']")
        cantidad_pdf = len(pdf)

        try:
            for i in pdf:

                nombre_archivo_pdf = i.text.strip()
               

                i.click()
                time.sleep(7)

                carpeta_descargas = os.path.expanduser('/Users/datos-lc/Downloads/')
                files = os.listdir(carpeta_descargas)
                if not files:
                        print("No se encontraron archivos en la carpeta de descargas.")
                        no_descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
                else:
                    name_descarga = pdf_files[0]
                    os.rename(os.path.join(carpeta_descargas, name_descarga), os.path.join(ruta_completa_contrato, nombre_archivo_pdf))
                    descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina, name_descarga, nombre_archivo_pdf])


                time.sleep(2)

        except Exception as e:
                    print("Error en la descarga de los enlaces de este contrato:", e)
                    print(nombre_archivo_pdf)
                    no_descargados.append([nombre_carpeta_year, nombre_carpeta_contrato, cantidad_pdf, expediente_excel, expediente_pagina])
                    #driver.quit()
                    time.sleep(2)
                    continue

        
df_no_descargados = pd.DataFrame(no_descargados, columns=["Year","Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina"])
df_descargados = pd.DataFrame(descargados, columns=["Year","Titulo_contrato", "Nro_pdfs_en_contrato", "Expediente_excel", "Expediente_pagina", "Nombre_archivo_descargod_original", "Nombre_nuevo_archivo_en_carpeta"])
                      





Carpeta creada: /Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2019
Error en la descarga de los enlaces de este contrato: [Errno 2] No such file or directory: '/Users/datos-lc/Downloads/(1) PROYECTO APNEA.pdf' -> '/Users/datos-lc/Documents/Colecciones ongoing/SEDENA/2019/Expediente 1862757 - ADQ. BIOLÓGICOS PARA DIAGNOSTICO Y TRATAMIENTO DE INMUNOLOGÍA CLÍNICA Y ALERGIA/CONV E135.pdf'
CONV E135.pdf


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.92)
Stacktrace:
0   chromedriver                        0x00000001084f9598 chromedriver + 4973976
1   chromedriver                        0x00000001084f0913 chromedriver + 4938003
2   chromedriver                        0x00000001080adcd6 chromedriver + 470230
3   chromedriver                        0x000000010808358c chromedriver + 296332
4   chromedriver                        0x000000010811af0f chromedriver + 917263
5   chromedriver                        0x000000010812fef6 chromedriver + 1003254
6   chromedriver                        0x0000000108115793 chromedriver + 894867
7   chromedriver                        0x00000001080e321f chromedriver + 688671
8   chromedriver                        0x00000001080e456e chromedriver + 693614
9   chromedriver                        0x00000001084b3bb7 chromedriver + 4688823
10  chromedriver   